# ANALYZING VENTILATOR DATA DG004

### Importing the necessary libraries and setting options

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from pprint import pprint
%matplotlib inline
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

### Clinical details and blood gases

In [ ]:
clin_details = pd.read_excel('/Users/guszti/Desktop/ventilation_data/data_grabber_patient_data.xlsx', 
                             sheetname = 'DG004')

In [ ]:
clin_details

In [ ]:
gestation = clin_details.ix[0, 1]; bw = clin_details.ix[2, 1]/1000; cw = clin_details.ix[4, 1]/1000
gestation

In [ ]:
bw

In [ ]:
cw 

In [ ]:
dg = clin_details.ix[5, 1].split(',')
dg

In [ ]:
comments = clin_details.ix[9, 1].split(',')
comments

#### Blood gases

In [ ]:
gases = pd.read_excel('/Users/guszti/Desktop/ventilation_data/data_grabber_gases.xlsx', sheetname = 'DG004')
gases = gases.T
gases.columns = gases.ix[0]
gases = gases.reset_index()

gases[[0,1,5,6,7,8,9,14,30]]


### Importing and cleaning up the ventilator data

### List and set the working directories

In [ ]:
%cd /Users/guszti/Desktop/ventilation_data/DG004
cwd = '/Users/guszti/Desktop/ventilation_data/DG004'
!ls -l

In [ ]:
fnames = ['CsvLogBase_2015-10-12_085111.498_', 'CsvLogBase_2015-10-12_175541.015_']

In [ ]:
dir_write = '/Users/guszti/Desktop/ventilation_data/Analyses/DG004'

In [ ]:
fnames2 = 'DG004_'

### Writing out clinical details in an Excel file

In [ ]:
writer = pd.ExcelWriter('%s/%s%s' % (dir_write, fnames2, 'clinical details.xlsx'))
clin_details.to_excel(writer,'clinical_details')
writer.save()

###  Importing slow measurement data

In [ ]:
slow_Measurement_1 = pd.read_csv('%s/%s%s' % (cwd, fnames[0], 'slow_Measurement.csv'), parse_dates = [[1, 2]])
slow_Measurement_1.index = slow_Measurement_1.Date_Time
slow_Measurement_2 = pd.read_csv('%s/%s%s' % (cwd, fnames[1], 'slow_Measurement.csv'), parse_dates = [[1, 2]])
slow_Measurement_2.index = slow_Measurement_2.Date_Time

In [ ]:
slow_Measurement = pd.concat([slow_Measurement_1, slow_Measurement_2])
del slow_Measurement_1
del slow_Measurement_2

In [ ]:
slow_Measurement.head()

### Renaming slow measurement data and clean-up

In [ ]:
slow_Measurement.info()

In [ ]:
columns = list(slow_Measurement.columns)
column_list = [(i, name) for i, name in enumerate(columns)]
column_list

In [ ]:
slow_Measurement[[38, 40]].head()

In [ ]:
slow_Measurement = slow_Measurement.drop(slow_Measurement[[38, 40]], axis = 1)

In [ ]:
columns = list(slow_Measurement.columns)
column_list = [(i, name) for i, name in enumerate(columns)]
column_list

In [ ]:
pprint([column_list[1], column_list[0] , column_list[9], column_list[4], column_list[2]])

In [ ]:
columns[1] = '5001|leak_pc'
columns[0] = '5001|MVspon_pc'
columns[9] = '5001|Iie'
columns[4] = '5001|Eie'
columns[2] = '5001|C20_Cdyn'

In [ ]:
columns

In [ ]:
new_names = [name.split(' ')[0][5:] for name in columns if name.startswith('5001')]
[(i, name) for i, name in enumerate(new_names)]

In [ ]:
columns = new_names  + ['date_time',  'rel_time', 'time_ms', ]
columns

In [ ]:
slow_Measurement.columns = columns
slow_Measurement.columns

In [ ]:
slow_Measurement.info()

### Calculating parameters / body weight kg

In [ ]:
slow_Measurement['VT_kg'] = slow_Measurement.VT / cw
slow_Measurement['VTmand_kg'] = slow_Measurement.VTmand / cw
slow_Measurement['VTspon_kg'] = slow_Measurement.VTspon / cw
slow_Measurement['VTi_kg'] = slow_Measurement.VTi / cw
slow_Measurement['VTe_kg'] = slow_Measurement.VTe / cw
slow_Measurement['VTimand_kg'] = slow_Measurement.VTimand / cw
slow_Measurement['VTemand_kg'] = slow_Measurement.VTemand / cw
slow_Measurement['VTispon_kg'] = slow_Measurement.VTispon / cw
slow_Measurement['VTespon_kg'] = slow_Measurement.VTespon / cw

In [ ]:
slow_Measurement['MV_kg'] = slow_Measurement.MV / cw
slow_Measurement['MVleak_kg'] = slow_Measurement.MVleak / cw
slow_Measurement['MVi_kg'] = slow_Measurement.MVi / cw
slow_Measurement['MVe_kg'] = slow_Measurement.MVe / cw
slow_Measurement['MVemand_kg'] = slow_Measurement.MVemand / cw
slow_Measurement['MVespon_kg'] = slow_Measurement.MVespon / cw

### Calcultating the I:E ratios

In [ ]:
slow_Measurement['IE_mand'] = slow_Measurement.Iie / slow_Measurement.Eie

In [ ]:
slow_Measurement.info()

### Resampling to remove half-empty rows

In [ ]:
slow_Measurement = slow_Measurement.resample('1S')

### Limiting the interval

In [ ]:
slow_Measurement.head()

### Calculate the recording period

In [ ]:
slow_Measurement.index[-1], slow_Measurement.index[0]

In [ ]:
recording_period = (slow_Measurement.index[-1] - slow_Measurement.index[0]).total_seconds()
recording_period

In [ ]:
recording_period_2 = (slow_Measurement.index[-1] - slow_Measurement.index[0])
print(recording_period_2)

### Writing out recording period in a text file¶

In [ ]:
fileout = open('%s/%s%s' % (dir_write, fnames2, 'slow_data_info.txt'), 'w')
fileout.write('Start point %s \n' % slow_Measurement.index[0])
fileout.write('End point %s \n' % slow_Measurement.index[-1])
fileout.write('Recording period: %s ' % recording_period_2)
fileout.write('\n\n\n')
fileout.close()

### Importing ventilator settings

In [ ]:
slow_Setting_1 = pd.read_csv('%s/%s%s' % (cwd, fnames[0], 'slow_Setting.csv'), parse_dates = [[1, 2]])
slow_Setting_1.index = slow_Setting_1.Date_Time
slow_Setting_2 = pd.read_csv('%s/%s%s' % (cwd, fnames[1], 'slow_Setting.csv'), parse_dates = [[1, 2]])
slow_Setting_2.index = slow_Setting_2.Date_Time

In [ ]:
slow_Setting = pd.concat([slow_Setting_1, slow_Setting_2])
del slow_Setting_1
del slow_Setting_2

In [ ]:
slow_Setting

In [ ]:
slow_Text_1 = pd.read_csv('%s/%s%s' % (cwd, fnames[0], 'slow_Text.csv'), parse_dates = [[1, 2]])
slow_Text_2 = pd.read_csv('%s/%s%s' % (cwd, fnames[1], 'slow_Text.csv'), parse_dates = [[1, 2]])
slow_Text_1.index = slow_Text_1.Date_Time
slow_Text_2.index = slow_Text_2.Date_Time

In [ ]:
slow_Text = pd.concat([slow_Text_1, slow_Text_2])
del slow_Text_1
del slow_Text_2

In [ ]:
slow_Text.index = range(len(slow_Text))
slow_Text

### 7. Filtering important slow settings

In [ ]:
set_Pinsp = slow_Setting.ix[slow_Setting.Id == 'Pinsp']
set_VTi = slow_Setting.ix[slow_Setting.Id == 'VTi']
set_Ti = slow_Setting.ix[slow_Setting.Id == 'Ti']
set_Te = slow_Setting.ix[slow_Setting.Id == 'Te']
set_Pmax = slow_Setting.ix[slow_Setting.Id == 'Pmax']
set_PEEP = slow_Setting.ix[slow_Setting.Id == 'PEEP']
set_RR = slow_Setting.ix[slow_Setting.Id == 'RR']
set_MAPhf = slow_Setting.ix[slow_Setting.Id == 'MAPhf']
set_VThf = slow_Setting.ix[slow_Setting.Id == 'VThf']
set_fhf = slow_Setting.ix[slow_Setting.Id == 'fhf']
pd.concat([set_Pinsp, set_VTi, set_Ti, set_Te, set_Pmax, set_PEEP, set_RR, set_MAPhf, set_VThf, set_fhf]).sort()

In [ ]:
slow_Text.ix[[0, 5]]

### Writing slow measurements and ventilator settings into .csv text files

In [ ]:
writer = pd.ExcelWriter('%s/%s%s' % (dir_write, fnames2, 'ventilator_settings.xlsx'))
slow_Setting.to_excel(writer,'Vent_Settings')
slow_Text.to_excel(writer,'Vent_Modes')
writer.save()

### Defining custom functions

In [ ]:
def pc5(x):
    return np.percentile(x, 5)

def pc25(x):
    return np.percentile(x, 25)

# slow_Measurement.resample('1min', how = 'median') - This code gives an Assertion error for reasons uncertain to me; 
# median cannot be calculated for all parameters at once.
# This also gives the median (almost) and it works
def pc50(x):
    return np.percentile(x, 50) 

def pc75(x):
    return np.percentile(x, 75)

def pc95(x):
    return np.percentile(x, 95)

In [ ]:
def par_stats(par):
    '''
    input: DataFrame containing aggregated data (e.g. 1-min medians etc...)
    This function return detailed statistics about the reduced data ;
    - number of data points in the aggregated data set, mean, standard deviation, median, mean absolute deviation,
      minium,5th centila, 25th centile, 75th centile, 95th centile, maximum of the time periods over the recording time 
    '''
    return (par.size, par.mean(), par.std(), par.median(), par.mad(), par.min(), par.quantile(0.05), 
            par.quantile(0.25), par.quantile(0.75), par.quantile(0.95), par.max())

### Calculating descriptive statistics on the slow data (1/sec) without resampling and writing them to files

### Fraction of inspired oxygen

In [ ]:
data = par_stats(slow_Measurement['FiO2'])
FiO2_stats = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
FiO2_stats['FiO2'] = 'FiO2'
FiO2_stats

### Pressures

In [ ]:
pressure_list = ['PIP', 'Pmean', 'PEEP', 'EIP', 'Pmin']
slow_Measurement_pressures = slow_Measurement[pressure_list]

In [ ]:
pressure_stats = []
for column in slow_Measurement_pressures.columns:
    data = par_stats(slow_Measurement_pressures[column])
    frame = DataFrame([data], columns = ['number_of_seconds', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['pressures'] = column
    pressure_stats.append(frame)

pressure_stats_all = pd.concat(pressure_stats)
pressure_stats_all

### Tidal volumes

In [ ]:
VT_list = ['VT_kg', 'VTi_kg', 'VTe_kg', 'VTmand_kg', 'VTspon_kg', 'VTimand_kg',  'VTemand_kg', 'VTispon_kg',
           'VTespon_kg']
slow_Measurement_VTs = slow_Measurement[VT_list]

In [ ]:
VT_stats = []
for column in slow_Measurement_VTs.columns:
    data = par_stats(slow_Measurement_VTs[column])
    frame = DataFrame([data], columns = ['number_of_seconds', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['tidal volumes'] = column
    VT_stats.append(frame)

VT_stats_all = pd.concat(VT_stats)
VT_stats_all

### Minute volumes

In [ ]:
MV_list = ['MV_kg', 'MVi_kg', 'MVe_kg', 'MVemand_kg','MVespon_kg', 'MVspon_pc', 'MVleak_kg']
slow_Measurement_MVs = slow_Measurement[MV_list]

In [ ]:
MV_stats = []
for column in slow_Measurement_MVs.columns:
    data = par_stats(slow_Measurement_MVs[column])
    frame = DataFrame([data], columns = ['number_of_seconds', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['minute volumes'] = column
    MV_stats.append(frame)

MV_stats_all = pd.concat(MV_stats)
MV_stats_all

### Respiratory rates

In [ ]:
RR_list = ['RR', 'RRmand', 'RRspon']
slow_Measurement_RRs = slow_Measurement[RR_list]

In [ ]:
RR_stats = []
for column in slow_Measurement_RRs.columns:
    data = par_stats(slow_Measurement_RRs[column])
    frame = DataFrame([data], columns = ['number_of_seconds', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['rates'] = column
    RR_stats.append(frame)

RR_stats_all = pd.concat(RR_stats)
RR_stats_all

## Leak

In [ ]:
leak_list = ['MVleak_kg', 'leak_pc',]
slow_Measurement_leaks = slow_Measurement[leak_list]

In [ ]:
leak_stats = []
for column in slow_Measurement_leaks.columns:
    data = par_stats(slow_Measurement_leaks[column])
    frame = DataFrame([data], columns = ['number_of_seconds', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['leak'] = column
    leak_stats.append(frame)

leak_stats_all = pd.concat(leak_stats)
leak_stats_all

### Device flow

In [ ]:
data = par_stats(slow_Measurement['FlowDev'])
flow_stats = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
flow_stats['FlowDev'] = 'FlowDev'
flow_stats

### I:E ratio of mandatory breaths

In [ ]:
data = par_stats(slow_Measurement['IE_mand'])
IE_stats = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
IE_stats['IE_mand'] = 'IE_mand'
IE_stats

### Compliance, resistance and time constant

In [ ]:
der_data_list = ['Cdyn', 'R', 'TC']
slow_Measurement_der_data = slow_Measurement[der_data_list]

In [ ]:
der_data_stats = []
for column in slow_Measurement_der_data.columns:
    data = par_stats(slow_Measurement_der_data[column])
    frame = DataFrame([data], columns = ['number_of_seconds', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['derived data'] = column
    der_data_stats.append(frame)

der_data_stats_all = pd.concat(der_data_stats)
der_data_stats_all

### Writing data to Excel file

In [ ]:
writer = pd.ExcelWriter('%s/%s%s' % (dir_write, fnames2, 'stats_1sec_data.xlsx'))
pressure_stats_all.to_excel(writer, 'pressures')
VT_stats_all.to_excel(writer, 'VTs')
MV_stats_all.to_excel(writer, 'MVs')
RR_stats_all.to_excel(writer, 'RRs')
leak_stats_all.to_excel(writer, 'leaks')
flow_stats.to_excel(writer, 'flow')
IE_stats.to_excel(writer, 'IE')
FiO2_stats.to_excel(writer, 'FiO2')
der_data_stats_all.to_excel(writer, 'derived data')
writer.save()

### Calculating statistics for 1-minute time windows and writing them to files

### Fraction of inspired oxygen

In [ ]:
slow_Measurement_FiO2_1min_median = slow_Measurement[['FiO2']].resample('1min', how = 'median')

In [ ]:
# descriptive statistics about 1-minute medians and showing the results
data = par_stats(slow_Measurement_FiO2_1min_median['FiO2'])
FiO2_1min_median_stats = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
FiO2_1min_median_stats['FiO2'] = 'FiO2'
FiO2_1min_median_stats

### Pressures

In [ ]:
slow_Measurement_pressures_1min_median = slow_Measurement[pressure_list].resample('1min', how = 'median')
slow_Measurement_pressures_1min_pc25 = slow_Measurement[pressure_list].resample('1min', how = pc25)
slow_Measurement_pressures_1min_pc75 = slow_Measurement[pressure_list].resample('1min', how = pc75)
slow_Measurement_pressures_1min_IQR = slow_Measurement_pressures_1min_pc75 - slow_Measurement_pressures_1min_pc25


In [ ]:
# descriptive statistics about 1-minute medians and showing the results
pressure_1min_median_stats = []
for column in slow_Measurement_pressures_1min_median.columns:
    data = par_stats(slow_Measurement_pressures_1min_median[column])
    frame = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['pressure'] = column
    pressure_1min_median_stats.append(frame)

pressure_1min_median_stats_all = pd.concat(pressure_1min_median_stats)
pressure_1min_median_stats_all

In [ ]:
# descriptive statistics about 1-minute IQRs and showing the results
pressure_1min_IQR_stats = []
for column in slow_Measurement_pressures_1min_IQR.columns:
    data = par_stats(slow_Measurement_pressures_1min_IQR[column])
    frame = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['pressure'] = column
    pressure_1min_IQR_stats.append(frame)

pressure_1min_IQR_stats_all = pd.concat(pressure_1min_IQR_stats)
pressure_1min_IQR_stats_all

### Tidal volumes

In [ ]:
slow_Measurement_VT_1min_median = slow_Measurement[VT_list].resample('1min', how = 'median')
slow_Measurement_VT_1min_pc25 = slow_Measurement[VT_list].resample('1min', how = pc25)
slow_Measurement_VT_1min_pc75 = slow_Measurement[VT_list].resample('1min', how = pc75)
slow_Measurement_VT_1min_IQR = slow_Measurement_VT_1min_pc75 - slow_Measurement_VT_1min_pc25

In [ ]:
# descriptive statistics about 1-minute medians and showing the results
VT_1min_median_stats = []
for column in slow_Measurement_VT_1min_median.columns:
    data = par_stats(slow_Measurement_VT_1min_median[column])
    frame = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['tidal volume'] = column
    VT_1min_median_stats.append(frame)

VT_1min_median_stats_all = pd.concat(VT_1min_median_stats)
VT_1min_median_stats_all

In [ ]:
# descriptive statistics about 1-minute IQRs and showing the results
VT_1min_IQR_stats = []
for column in slow_Measurement_VT_1min_IQR.columns:
    data = par_stats(slow_Measurement_VT_1min_IQR[column])
    frame = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['tidal volume'] = column
    VT_1min_IQR_stats.append(frame)

VT_1min_IQR_stats_all = pd.concat(VT_1min_IQR_stats)
VT_1min_IQR_stats_all

### Minute volumes

In [ ]:
slow_Measurement_MV_1min_median = slow_Measurement[MV_list].resample('1min', how = 'median')
slow_Measurement_MV_1min_pc25 = slow_Measurement[MV_list].resample('1min', how = pc25)
slow_Measurement_MV_1min_pc75 = slow_Measurement[MV_list].resample('1min', how = pc75)
slow_Measurement_MV_1min_IQR = slow_Measurement_MV_1min_pc75 - slow_Measurement_MV_1min_pc25

In [ ]:
# descriptive statistics about 1-minute medians and showing the results
MV_1min_median_stats = []
for column in slow_Measurement_MV_1min_median.columns:
    data = par_stats(slow_Measurement_MV_1min_median[column])
    frame = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['minute volume'] = column
    MV_1min_median_stats.append(frame)

MV_1min_median_stats_all = pd.concat(MV_1min_median_stats)
MV_1min_median_stats_all

In [ ]:
# descriptive statistics about 1-minute IQRs and showing the results
MV_1min_IQR_stats = []
for column in slow_Measurement_MV_1min_IQR.columns:
    data = par_stats(slow_Measurement_MV_1min_IQR[column])
    frame = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['minute volume'] = column
    MV_1min_IQR_stats.append(frame)

MV_1min_IQR_stats_all = pd.concat(MV_1min_IQR_stats)
MV_1min_IQR_stats_all

### Respiratory rates

In [ ]:
slow_Measurement_RR_1min_median = slow_Measurement[RR_list].resample('1min', how = 'median')
slow_Measurement_RR_1min_pc25 = slow_Measurement[RR_list].resample('1min', how = pc25)
slow_Measurement_RR_1min_pc75 = slow_Measurement[RR_list].resample('1min', how = pc75)
slow_Measurement_RR_1min_IQR = slow_Measurement_RR_1min_pc75 - slow_Measurement_RR_1min_pc25

In [ ]:
# descriptive statistics about 1-minute medians and showing the results
RR_1min_median_stats = []
for column in slow_Measurement_RR_1min_median.columns:
    data = par_stats(slow_Measurement_RR_1min_median[column])
    frame = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['respiratory rate'] = column
    RR_1min_median_stats.append(frame)

RR_1min_median_stats_all = pd.concat(RR_1min_median_stats)
RR_1min_median_stats_all

In [ ]:
# descriptive statistics about 1-minute IQRs and showing the results
RR_1min_IQR_stats = []
for column in slow_Measurement_RR_1min_IQR.columns:
    data = par_stats(slow_Measurement_RR_1min_IQR[column])
    frame = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['respiratory rate'] = column
    RR_1min_IQR_stats.append(frame)

RR_1min_IQR_stats_all = pd.concat(RR_1min_IQR_stats)
RR_1min_IQR_stats_all

### Leak

In [ ]:
slow_Measurement_leak_1min_median = slow_Measurement[leak_list].resample('1min', how = 'median')
slow_Measurement_leak_1min_pc25 = slow_Measurement[leak_list].resample('1min', how = pc25)
slow_Measurement_leak_1min_pc75 = slow_Measurement[leak_list].resample('1min', how = pc75)
slow_Measurement_leak_1min_IQR = slow_Measurement_leak_1min_pc75 - slow_Measurement_leak_1min_pc25

In [ ]:
# descriptive statistics about 1-minute medians and showing the results
leak_1min_median_stats = []
for column in slow_Measurement_leak_1min_median.columns:
    data = par_stats(slow_Measurement_leak_1min_median[column])
    frame = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['leak parameter'] = column
    leak_1min_median_stats.append(frame)

leak_1min_median_stats_all = pd.concat(leak_1min_median_stats)
leak_1min_median_stats_all

In [ ]:
# descriptive statistics about 1-minute IQRs and showing the results
leak_1min_IQR_stats = []
for column in slow_Measurement_leak_1min_IQR.columns:
    data = par_stats(slow_Measurement_leak_1min_IQR[column])
    frame = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['leak parameter'] = column
    leak_1min_IQR_stats.append(frame)

leak_1min_IQR_stats_all = pd.concat(leak_1min_IQR_stats)
leak_1min_IQR_stats_all

### Device flow

In [ ]:
slow_Measurement_flow_1min_median = slow_Measurement[['FlowDev',]].resample('1min', how = 'median')
slow_Measurement_flow_1min_pc25 = slow_Measurement[['FlowDev',]].resample('1min', how = pc25)
slow_Measurement_flow_1min_pc75 = slow_Measurement[['FlowDev']].resample('1min', how = pc75)
slow_Measurement_flow_1min_IQR = slow_Measurement_flow_1min_pc75 - slow_Measurement_flow_1min_pc25

In [ ]:
# descriptive statistics about 1-minute medians and showing the results
data = par_stats(slow_Measurement_flow_1min_median['FlowDev'])
flow_1min_median_stats = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
flow_1min_median_stats['flow'] = 'Device flow'
flow_1min_median_stats

In [ ]:
# descriptive statistics about 1-minute IQRs and showing the results
data = par_stats(slow_Measurement_flow_1min_IQR['FlowDev'])
flow_1min_IQR_stats = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
flow_1min_IQR_stats['flow'] = 'Device flow'
flow_1min_IQR_stats

### I:E ratio of mandatory breaths

In [ ]:
slow_Measurement_IE_1min_median = slow_Measurement[['IE_mand',]].resample('1min', how = 'median')
slow_Measurement_IE_1min_pc25 = slow_Measurement[['IE_mand',]].resample('1min', how = pc25)
slow_Measurement_IE_1min_pc75 = slow_Measurement[['IE_mand']].resample('1min', how = pc75)
slow_Measurement_IE_1min_IQR = slow_Measurement_IE_1min_pc75 - slow_Measurement_IE_1min_pc25

In [ ]:
# descriptive statistics about 1-minute medians and showing the results
data = par_stats(slow_Measurement_IE_1min_median['IE_mand'])
IE_1min_median_stats = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
IE_1min_median_stats['I:E ratio'] = 'I:E ratio of mandatory breaths'
IE_1min_median_stats

In [ ]:
# descriptive statistics about 1-minute IQRs and showing the results
data = par_stats(slow_Measurement_IE_1min_IQR['IE_mand'])
IE_1min_IQR_stats = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
IE_1min_IQR_stats['I:E ratio'] = 'I:E ratio of mandatory breaths'
IE_1min_IQR_stats

### Compliance, resistance and time constant

In [ ]:
slow_Measurement_der_data_1min_median = slow_Measurement[der_data_list].resample('1min', how = 'median')
slow_Measurement_der_data_1min_pc25 = slow_Measurement[der_data_list].resample('1min', how = pc25)
slow_Measurement_der_data_1min_pc75 = slow_Measurement[der_data_list].resample('1min', how = pc75)
slow_Measurement_der_data_1min_IQR = slow_Measurement_der_data_1min_pc75 - slow_Measurement_der_data_1min_pc25

In [ ]:
# descriptive statistics about 1-minute medians and showing the results
der_data_1min_median_stats = []
for column in slow_Measurement_der_data_1min_median.columns:
    data = par_stats(slow_Measurement_der_data_1min_median[column])
    frame = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['derived data'] = column
    der_data_1min_median_stats.append(frame)
der_data_1min_median_stats_all = pd.concat(der_data_1min_median_stats)
der_data_1min_median_stats_all

In [ ]:
# descriptive statistics about 1-minute medians and showing the results
der_data_1min_IQR_stats = []
for column in slow_Measurement_der_data_1min_IQR.columns:
    data = par_stats(slow_Measurement_der_data_1min_IQR[column])
    frame = DataFrame([data], columns = ['number_of_minutes', 'mean', 'stdev', 'median', 'mad',
                'min', '5pc', '25pc', '75pc', '95pc', 'max' ])
    frame['derived data'] = column
    der_data_1min_IQR_stats.append(frame)

der_data_1min_IQR_stats_all = pd.concat(der_data_1min_IQR_stats)
der_data_1min_IQR_stats_all

### Writing data into an Excel file

In [ ]:
writer = pd.ExcelWriter('%s/%s%s' % (dir_write, fnames2, 'stats_1min_windows.xlsx'))
pressure_1min_median_stats_all.to_excel(writer, 'pressure_medians')
pressure_1min_IQR_stats_all.to_excel(writer, 'pressure_IQRs')
VT_1min_median_stats_all.to_excel(writer, 'VT_medians')
VT_1min_IQR_stats_all.to_excel(writer, 'VT_IQRs')
MV_1min_median_stats_all.to_excel(writer, 'MV_medians')
MV_1min_IQR_stats_all.to_excel(writer, 'MV_IQRs')
RR_1min_median_stats_all.to_excel(writer, 'RR_medians')
RR_1min_IQR_stats_all.to_excel(writer, 'RR_IQRs')
leak_1min_median_stats_all.to_excel(writer, 'leak_medians')
leak_1min_IQR_stats_all.to_excel(writer, 'leak_IQRs')
flow_1min_median_stats.to_excel(writer, 'flow_medians')
flow_1min_IQR_stats.to_excel(writer, 'flow_IQRs')
IE_1min_median_stats.to_excel(writer, 'IE_medians')
IE_1min_IQR_stats.to_excel(writer, 'IE_IQRs')
FiO2_1min_median_stats.to_excel(writer, 'FiO2_medians')
der_data_1min_median_stats_all.to_excel(writer, 'derived_data_medians')
der_data_1min_IQR_stats_all.to_excel(writer, 'derived_data_IQRs')
writer.save()

# Visualization

### FiO2

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_FiO2_1min_median.plot(ax = ax1, color = 'blue', ylim = [-5, 100])
ax1.set_title('FiO2 - 1-minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black', )
ax1.set_ylabel('%', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')

### Pressures

### Boxplot of of the medians and IQR

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 35)
fig.suptitle('Boxplots of 1-minute medians \n\n Pressures', color = 'black', size = 22, weight = 'bold')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=0.3, hspace=0.3)
for i, column in enumerate(slow_Measurement_pressures_1min_median.columns):
    ax = fig.add_subplot(8, 3, i+1)
    ax.boxplot(slow_Measurement_pressures_1min_median[column], )
    ax.set_title(column, size = 16, color = 'black')
    ax.set_xlabel('values', size = 14, color = 'black')
    ax.set_ylabel('number', size = 14, color = 'black');

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'boxplot_pressures_1min_medians.pdf'), dpi=100, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 35)
fig.suptitle('Boxplots of 1-minute IQRs \n\n Pressures', color = 'black', size = 22, weight = 'bold')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=0.3, hspace=0.3)
for i, column in enumerate(slow_Measurement_pressures_1min_IQR.columns):
    ax = fig.add_subplot(8, 3, i+1)
    ax.boxplot(slow_Measurement_pressures_1min_IQR[column], )
    ax.set_title(column, size = 16, color = 'black')
    ax.set_xlabel('values', size = 14, color = 'black')
    ax.set_ylabel('number', size = 14, color = 'black');
    
fig.savefig('%s/%s%s' % (dir_write, fnames2, 'boxplot_pressures_1min_IQRs.pdf'), dpi=100, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

### Plotting PIP, Pmean, PEEP versus time - no resampling (this takes some time to compute)

### Plotting PIP, Pmean and PEEP medians and IQRs (over 1-min time windows) versus time

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_pressures_1min_median['PIP'].plot(ax = ax1, color = 'red', 
                                        ylim = [0, (slow_Measurement_pressures_1min_median['PIP'].max() + 5)] );
slow_Measurement_pressures_1min_median['Pmean'].plot(ax = ax1, color = 'green', 
                                        ylim = [0, (slow_Measurement_pressures_1min_median['PIP'].max() + 5)] );
slow_Measurement_pressures_1min_median['PEEP'].plot(ax = ax1, color = 'blue', 
                                        ylim = [0, (slow_Measurement_pressures_1min_median['PIP'].max() + 5)] );
ax1.set_title('Pressures - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Pressure (mbar)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['PIP', 'MAP', 'PEEP']);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_pressures_1min_median['Pmean'].plot(ax = ax1, color = 'green', 
                                        ylim = [0, (slow_Measurement_pressures_1min_median['Pmean'].max() + 5)] );
ax1.set_title('Pressures - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Pressure (mbar)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend([ 'MAP']);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_pressures_1min_IQR['PIP'].plot(ax = ax1, color = 'red', 
                                        ylim = [0, (slow_Measurement_pressures_1min_IQR['PIP'].max() + 1)] );
slow_Measurement_pressures_1min_IQR['Pmean'].plot(ax = ax1, color = 'green', 
                                        ylim = [0, (slow_Measurement_pressures_1min_IQR['PIP'].max() + 1)] );
slow_Measurement_pressures_1min_IQR['PEEP'].plot(ax = ax1, color = 'blue', 
                                        ylim = [0, (slow_Measurement_pressures_1min_IQR['PIP'].max() + 1)] );
ax1.set_title('Pressures - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Pressure (mbar)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['PIP', 'MAP', 'PEEP']);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_pressures_1min_IQR['Pmean'].plot(ax = ax1, color = 'green', 
                                        ylim = [0, (slow_Measurement_pressures_1min_IQR['Pmean'].max() + 5)] );
ax1.set_title('Pressures - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Pressure (mbar)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend([ 'MAP']);

### EIP, PIP versus time -  not resampled

### EIP and  PIP medians and IQRs versus time

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_pressures_1min_median['EIP'].plot(ax = ax1, color = 'red', 
                                        ylim = [0, (slow_Measurement_pressures_1min_median['PIP'].max() + 1)] );
slow_Measurement_pressures_1min_median['PIP'].plot(ax = ax1, color = 'green', 
                                        ylim = [0, (slow_Measurement_pressures_1min_median['PIP'].max() + 1)] );
ax1.set_title('End inspiratory pressure and peak inspiratory pressure - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Pressure (mbar)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['EIP', 'PIP']);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_pressures_1min_IQR['EIP'].plot(ax = ax1, color = 'red', 
                                        ylim = [0, (slow_Measurement_pressures_1min_IQR['PIP'].max() + 1)] );
slow_Measurement_pressures_1min_IQR['PIP'].plot(ax = ax1, color = 'green', 
                                        ylim = [0, (slow_Measurement_pressures_1min_IQR['PIP'].max() + 1)] );
ax1.set_title('End inspiratory pressure and peak inspiratory pressure - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Pressure (mbar)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['EIP', 'PIP']);

### PEEP, Pmin versus time - not resampled

### PEEP, Pmin medians and IQRs versus time

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_pressures_1min_median['PEEP'].plot(ax = ax1, color = 'red', 
                                        ylim = [-10, (slow_Measurement_pressures_1min_median['PEEP'].max() + 3)] );
slow_Measurement_pressures_1min_median['Pmin'].plot(ax = ax1, color = 'green', 
                                        ylim = [-10, (slow_Measurement_pressures_1min_median['PEEP'].max() + 3)] );
ax1.set_title('Positive End Expiratory Pressure and Minimum Pressure - 1-min medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Pressure (mbar)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['PEEP', 'Pmin']);

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'PEEP_Pmin.pdf'), dpi=300, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_pressures_1min_IQR['PEEP'].plot(ax = ax1, color = 'red', 
                                        ylim = [0, (slow_Measurement_pressures_1min_IQR['PEEP'].max() + 1)] );
slow_Measurement_pressures_1min_IQR['Pmin'].plot(ax = ax1, color = 'green', 
                                        ylim = [0, (slow_Measurement_pressures_1min_IQR['PEEP'].max() + 1)] );
ax1.set_title('Positive End Expiratory Pressure and Minimum Pressure - 1-min IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Pressure (mbar)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['PEEP', 'Pmin']);

### Tidal Volumes

### Boxplot of the medians and IQRs

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 35)
fig.suptitle('Boxplots of 1-minute medians \n\n Tidal Volumes', color = 'black', size = 22, weight = 'bold')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=0.3, hspace=0.3)
for i, column in enumerate(slow_Measurement_VT_1min_median.columns):
    ax = fig.add_subplot(8, 3, i+1)
    ax.boxplot(slow_Measurement_VT_1min_median[column], )
    ax.set_title(column, size = 16, color = 'black')
    ax.set_xlabel('values', size = 14, color = 'black')
    ax.set_ylabel('number', size = 14, color = 'black');
    
fig.savefig('%s/%s%s' % (dir_write, fnames2, 'boxplot_VT_1min_medians.pdf'), dpi=100, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 35)
fig.suptitle('Boxplots of 1-minute IQRs \n\n Tidal Volumes', color = 'black', size = 22, weight = 'bold')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=0.3, hspace=0.3)
for i, column in enumerate(slow_Measurement_VT_1min_IQR.columns):
    ax = fig.add_subplot(8, 3, i+1)
    ax.boxplot(slow_Measurement_VT_1min_IQR[column], )
    ax.set_title(column, size = 16, color = 'black')
    ax.set_xlabel('values', size = 14, color = 'black')
    ax.set_ylabel('number', size = 14, color = 'black');
    
fig.savefig('%s/%s%s' % (dir_write, fnames2, 'boxplot_VT_1min_IQRs.pdf'), dpi=100, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

### Inspiratory and expiratory tidal volumes

###### *VT* is leakage compensated, VTi and VTe are not leakage compensated

#### not resampled

#### 1-minute medians

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_VT_1min_median['VT_kg'].plot(ax = ax1, color = 'blue', ylim = [0, (slow_Measurement_VT_1min_median['VTi_kg'].max() + 5)])
slow_Measurement_VT_1min_median['VTi_kg'].plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_VT_1min_median['VTi_kg'].max() + 5)],)
slow_Measurement_VT_1min_median['VTe_kg'].plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_VT_1min_median['VTi_kg'].max() + 5)],)
ax1.set_title('Tidal Volumes - 1-minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (mL/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['VT', 'VTi', 'VTe' ]);

##### 1-min IQRs

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_VT_1min_IQR['VT_kg'].plot(ax = ax1, color = 'blue', ylim = [0, (slow_Measurement_VT_1min_IQR['VTi_kg'].max() + 5)])
slow_Measurement_VT_1min_IQR['VTi_kg'].plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_VT_1min_IQR['VTi_kg'].max() + 5)],)
slow_Measurement_VT_1min_IQR['VTe_kg'].plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_VT_1min_IQR['VTi_kg'].max() + 5)],)
ax1.set_title('Tidal Volumes - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (mL/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['VT', 'VTi', 'VTe' ]);

### Mandatory (supported) and spontaneous tidal volumes 

###### Both these are leak-corrected

#### not resampled

#### 1-min medians

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_VT_1min_median.VTmand_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_VT_1min_median.VTmand_kg.max() * 1.2)])
slow_Measurement_VT_1min_median.VTspon_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_VT_1min_median.VTmand_kg.max() * 1.2)],)
ax1.set_title('Tidal Volume - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (mL/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['VT_mand', 'VT_spon', ]);

#### 1-min IQRs

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_VT_1min_IQR.VTmand_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_VT_1min_IQR.VTmand_kg.max() * 1.2)])
slow_Measurement_VT_1min_IQR.VTspon_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_VT_1min_IQR.VTmand_kg.max() * 1.2)],)
ax1.set_title('Tidal Volume - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (mL/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['VT_mand', 'VT_spon', ]);

###  Inspiratory and expiratory tidal volumes

### inspiratory

##### VTimand and VTispon are not leak corrected

##### not resampled

##### 1-min medians

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_VT_1min_median.VTimand_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_VT_1min_median.VTimand_kg.max() * 1.2)])
slow_Measurement_VT_1min_median.VTispon_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_VT_1min_median.VTimand_kg.max() * 1.2)], )
ax1.set_title('Inspiratory Tidal Volumes - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (mL/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['VTimand', 'VTispon', ]);

##### 1-min IQRs

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_VT_1min_IQR.VTimand_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_VT_1min_IQR.VTimand_kg.max() * 0.2)])
slow_Measurement_VT_1min_IQR.VTispon_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_VT_1min_IQR.VTimand_kg.max() * 0.2)], )
ax1.set_title('Inspiratory Tidal Volumes - 1-min IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (mL/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['VTimand', 'VTispon', ]);

### expiratory

##### VTemand and VTiespon are not leak corrected

##### not resampled

##### 1-min medians

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_VT_1min_median.VTemand_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_VT_1min_median.VTespon_kg.max() * 1.2)])
slow_Measurement_VT_1min_median.VTespon_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_VT_1min_median.VTespon_kg.max() * 1.2)], )
ax1.set_title('Mandatory and Spontaneous Expiratory Tidal Volumes', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (mL/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['VTemand', 'VTespon', ]);

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'VT_emand_espon.pdf'), dpi=300, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

##### 1-min IQRs 

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_VT_1min_IQR.VTemand_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_VT_1min_IQR.VTemand_kg.max() * 1.2)])
slow_Measurement_VT_1min_IQR.VTespon_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_VT_1min_IQR.VTemand_kg.max() * 1.2)], )
ax1.set_title('Expiratory Tidal Volumes - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (mL/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['VTemand', 'VTespon', ]);

### Mandatory tidal volumes

##### VTmand is leak-corrected, VTimand and VTemand are not leak-corrected

##### not resampled

##### 1-min medians

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_VT_1min_median.VTimand_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_VT_1min_median.VTimand_kg.max() * 1.2)])
slow_Measurement_VT_1min_median.VTmand_kg.plot(ax = ax1, color = 'blue', ylim = [0, (slow_Measurement_VT_1min_median.VTimand_kg.max() * 1.2)])
slow_Measurement_VT_1min_median.VTemand_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_VT_1min_median.VTimand_kg.max() * 1.2)], )
ax1.set_title('Mandatory Tidal Volumes - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (mL/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['VTimand', 'VTmand', 'VTemand', ]);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_VT_1min_median.VTimand_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_VT_1min_median.VTimand_kg.max() * 1.2)])
slow_Measurement_VT_1min_median.VTemand_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_VT_1min_median.VTimand_kg.max() * 1.2)], )
ax1.set_title('Mandatory Inspiratory and Expiratory Tidal Volumes', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (mL/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['VTimand', 'VTemand', ]);

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'VT_imand_emand.pdf'), dpi=300, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

##### 1-min IQRs

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_VT_1min_IQR.VTimand_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_VT_1min_IQR.VTimand_kg.max() * 1.2)])
slow_Measurement_VT_1min_IQR.VTmand_kg.plot(ax = ax1, color = 'blue', ylim = [0, (slow_Measurement_VT_1min_IQR.VTmand_kg.max() * 1.2)])
slow_Measurement_VT_1min_IQR.VTemand_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_VT_1min_IQR.VTemand_kg.max() * 1.2)], )
ax1.set_title('Mandatory Tidal Volumes - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (mL/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['VTimand', 'VTmand', 'VTemand', ]);

### Spontaneous tidal volumes

##### VTspon is leak-corrected, VTispon and VTesponare not leak-corrected

#### not resampled

#### 1-min medians

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_VT_1min_median.VTispon_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_VT_1min_median.VTispon_kg.max() * 1.2)])
slow_Measurement_VT_1min_median.VTspon_kg.plot(ax = ax1, color = 'blue', ylim = [0, (slow_Measurement_VT_1min_median.VTispon_kg.max() * 1.2)])
slow_Measurement_VT_1min_median.VTespon_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_VT_1min_median.VTispon_kg.max() * 1.2)], )
ax1.set_title('Spontaneous Tidal Volumes - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (mL/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['VTispon', 'VTspon', 'VTespon', ]);

##### 1-min IQRs

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_VT_1min_IQR.VTispon_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_VT_1min_IQR.VTispon_kg.max() * 1.2)])
slow_Measurement_VT_1min_IQR.VTspon_kg.plot(ax = ax1, color = 'blue', ylim = [0, (slow_Measurement_VT_1min_IQR.VTispon_kg.max() * 1.2)])
slow_Measurement_VT_1min_IQR.VTespon_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_VT_1min_IQR.VTispon_kg.max() * 1.2)], )
ax1.set_title('Spontaneous Tidal Volumes - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (mL/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['VTispon', 'VTspon', 'VTespon', ]);

## Minute Volumes

### 1/sec data, no resampling

### Boxplot of of the medians and IQRs over 1-minute time windows

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 35)
fig.suptitle('Boxplots of 1-minute medians \n\n Minute Volumes', color = 'black', size = 22, weight = 'bold')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=0.3, hspace=0.3)
for i, column in enumerate(slow_Measurement_MV_1min_median.columns):
    ax = fig.add_subplot(8, 3, i+1)
    ax.boxplot(slow_Measurement_MV_1min_median[column], )
    ax.set_title(column, size = 16, color = 'black')
    ax.set_xlabel('values', size = 14, color = 'black')
    ax.set_ylabel('number', size = 14, color = 'black');
    
fig.savefig('%s/%s%s' % (dir_write, fnames2, 'boxplot_MV_1min_medians.pdf'), dpi=100, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 35)
fig.suptitle('Boxplots of 1-minute IQRs \n\n Minute Volumes', color = 'black', size = 22, weight = 'bold')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=0.3, hspace=0.3)
for i, column in enumerate(slow_Measurement_MV_1min_IQR.columns):
    ax = fig.add_subplot(8, 3, i+1)
    ax.boxplot(slow_Measurement_MV_1min_IQR[column], )
    ax.set_title(column, size = 16, color = 'black')
    ax.set_xlabel('values', size = 14, color = 'black')
    ax.set_ylabel('number', size = 14, color = 'black');
    
fig.savefig('%s/%s%s' % (dir_write, fnames2, 'boxplot_MV_1min_IQR.pdf'), dpi=100, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

### Calculating mandatory (supported) and spontaneous minute volumes over 1-minute time windows

##### 1-min medians

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_MV_1min_median.MV_kg.plot(ax = ax1, color = 'blue', ylim = [0, (slow_Measurement_MV_1min_median.MV_kg.max() * 1.2)])
slow_Measurement_MV_1min_median.MVemand_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_MV_1min_median.MV_kg.max() * 1.2)]) 
slow_Measurement_MV_1min_median.MVespon_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_MV_1min_median.MV_kg.max() * 1.2)])
ax1.set_title('Mandatory and Spontaneous Expiratory Minute Volume - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Minute Volume (L/min/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['MV', 'MVemand', 'MVespon' ]);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_MV_1min_median.MVemand_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_MV_1min_median.MV_kg.max() * 1.2)]) 
slow_Measurement_MV_1min_median.MVespon_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_MV_1min_median.MV_kg.max() * 1.2)])
ax1.set_title('Mandatory and Spontaneous Expiratory Minute Volume', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Minute Volume (L/min/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend([ 'MVemand', 'MVespon' ]);

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'minute_volumes.pdf'), dpi=300, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_MV_1min_median.MV_kg.plot(ax = ax1, color = 'blue', ylim = [0, 0.5])
slow_Measurement_MV_1min_median.MVemand_kg.plot(ax = ax1, color = 'red', ylim = [0, 0.5]) 
slow_Measurement_MV_1min_median.MVespon_kg.plot(ax = ax1, color = 'green', ylim = [0, 0.5])
ax1.set_title('Mandatory and Spontaneous Expiratory Minute Volume - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Minute Volume (L/min/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['MV', 'MVemand', 'MVespon' ]);

##### 1-min IQRs

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_MV_1min_IQR.MVemand_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_MV_1min_IQR.MVemand_kg.max() * 1.2)]) 
slow_Measurement_MV_1min_IQR.MVespon_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_MV_1min_IQR.MVespon_kg.max() * 1.2)])
ax1.set_title('Mandatory and Spontaneous Expiratory Minute Volume - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Minute Volume (L/min/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['MVemand', 'MVespon' ]);

### Calculating inspiratory and expiratory minute volumes over 1-min time windows

##### 1-min medians

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_MV_1min_median.MV_kg.plot(ax = ax1, color = 'blue', ylim = [0, (slow_Measurement_MV_1min_median.MVi_kg.max() * 1.2)])
slow_Measurement_MV_1min_median.MVi_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_MV_1min_median.MVi_kg.max() * 1.2)])
slow_Measurement_MV_1min_median.MVe_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_MV_1min_median.MVi_kg.max() * 1.2)])
ax1.set_title('Inspiratory and Expiratory Minute Volume - 1-minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel(' Minute Volume (L/min/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['MV', 'MVi', 'MVe', ]);

##### 1-min IQRs

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_MV_1min_IQR.MV_kg.plot(ax = ax1, color = 'blue', ylim = [0, (slow_Measurement_MV_1min_IQR.MVi_kg.max() * 1.2)])
slow_Measurement_MV_1min_IQR.MVi_kg.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_MV_1min_IQR.MVi_kg.max() * 1.2)])
slow_Measurement_MV_1min_IQR.MVe_kg.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_MV_1min_IQR.MVi_kg.max() * 1.2)])
ax1.set_title('Inspiratory and Expiratory Minute Volume - 1-min IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel(' Minute Volume (L/min/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['MV', 'MVi', 'MVe', ]);

### Percentage of spontaneaous minute volume ( minute volume from unsupported breaths)

##### 1-min medians

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_MV_1min_median.MVspon_pc.plot(ax = ax1, color = 'blue', 
                                     ylim = [0, (slow_Measurement_MV_1min_median.MVspon_pc.max() * 1.2)])
ax1.set_title('Percentage of spontaneous minute volume - 1-minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black', )
ax1.set_ylabel('%', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend('MVspon%');

##### 1-min IQRs

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_MV_1min_IQR.MVspon_pc.plot(ax = ax1, color = 'blue', 
                                     ylim = [0, (slow_Measurement_MV_1min_IQR.MVspon_pc.max() * 1.2)])
ax1.set_title('Percentage of spontaneous minute volume - 1-minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black', )
ax1.set_ylabel('%', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend('MVspon%');

### Leak minute volume

##### 1-min medians

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_MV_1min_median.MVleak_kg.plot(ax = ax1, color = 'blue', ylim = [0, (slow_Measurement_MV_1min_median.MVleak_kg.max() * 1.2)])
ax1.set_title('Leak minute volume - 1-minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black', )
ax1.set_ylabel('L/min/kg', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend('MVleak');

##### 1-min IQRs

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_MV_1min_IQR.MVleak_kg.plot(ax = ax1, color = 'blue', ylim = [0, 0.5])
ax1.set_title('Leak minute volume - 1-minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black', )
ax1.set_ylabel('L/min/kg', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend('MVleak');

## Respiratory rates

### 1/sec data, no resampling

### Boxplot of of the medians and IQRs

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 35)
fig.suptitle('Boxplots of 1-minute medians \n\n Respiratory rates', color = 'black', size = 22, weight = 'bold')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=0.3, hspace=0.3)
for i, column in enumerate(slow_Measurement_RR_1min_median.columns):
    ax = fig.add_subplot(8, 3, i+1)
    ax.boxplot(slow_Measurement_RR_1min_median[column], )
    ax.set_title(column, size = 16, color = 'black')
    ax.set_xlabel('values', size = 14, color = 'black')
    ax.set_ylabel('number', size = 14, color = 'black');
    
fig.savefig('%s/%s%s' % (dir_write, fnames2, 'boxplot_RR_1min_medians.pdf'), dpi=100, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 35)
fig.suptitle('Boxplots of 1-minute IQRs \n\n Respiratory rates', color = 'black', size = 22, weight = 'bold')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=0.3, hspace=0.3)
for i, column in enumerate(slow_Measurement_RR_1min_IQR.columns):
    ax = fig.add_subplot(8, 3, i+1)
    ax.boxplot(slow_Measurement_RR_1min_IQR[column], )
    ax.set_title(column, size = 16, color = 'black')
    ax.set_xlabel('values', size = 14, color = 'black')
    ax.set_ylabel('number', size = 14, color = 'black');
    
fig.savefig('%s/%s%s' % (dir_write, fnames2, 'boxplot_RR_1min_IQRs.pdf'), dpi=100, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

### Calculating and plotting supported and spontaneous respiratory rates over 1-minute time windows

#### 1-min medians

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_RR_1min_median.RRmand.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_RR_1min_median.RRmand.max() * 1.2 )])
slow_Measurement_RR_1min_median.RRspon.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_RR_1min_median.RRmand.max() * 1.2)])
ax1.set_title('Respiratory rate - 1-minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Breaths / min', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend([ 'RRmand', 'RRspon']);

##### IQRs

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_RR_1min_IQR.RRmand.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_RR_1min_IQR.RRmand.max() * 1.2 )])
slow_Measurement_RR_1min_IQR.RRspon.plot(ax = ax1, color = 'green', ylim = [0, (slow_Measurement_RR_1min_IQR.RRspon.max() * 1.2)])
ax1.set_title('Respiratory rate - IQRs over 1-minute windows', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Breaths / min', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend([ 'RRmand', 'RRspon']);

## Leak

#### Not resampled, 1/sec data

#### Boxplot of the 1-min medians and IQRs

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 35)
fig.suptitle('Boxplots of 1-minute medians \n\n Leak Values', color = 'black', size = 22, weight = 'bold')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=0.3, hspace=0.3)
for i, column in enumerate(slow_Measurement_leak_1min_median.columns):
    ax = fig.add_subplot(8, 3, i+1)
    ax.boxplot(slow_Measurement_leak_1min_median[column], )
    ax.set_title(column, size = 16, color = 'black')
    ax.set_xlabel('values', size = 14, color = 'black')
    ax.set_ylabel('number', size = 14, color = 'black');
    
fig.savefig('%s/%s%s' % (dir_write, fnames2, 'boxplot_leak_1min_medians.pdf'), dpi=100, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 35)
fig.suptitle('Boxplots of 1-minute IQRs \n\n Leak Values', color = 'black', size = 22, weight = 'bold')
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=0.3, hspace=0.3)
for i, column in enumerate(slow_Measurement_leak_1min_IQR.columns):
    ax = fig.add_subplot(8, 3, i+1)
    ax.boxplot(slow_Measurement_leak_1min_IQR[column], )
    ax.set_title(column, size = 16, color = 'black')
    ax.set_xlabel('values', size = 14, color = 'black')
    ax.set_ylabel('number', size = 14, color = 'black');
    
fig.savefig('%s/%s%s' % (dir_write, fnames2, 'boxplot_leak_1min_IQRs.pdf'), dpi=100, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='pdf',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)

### Resampled over 1-min time windows

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_leak_1min_median.leak_pc.plot(ax = ax1, color = 'blue', 
                                   ylim = [0, (slow_Measurement_leak_1min_median.leak_pc.max() * 1.2)], )
ax1.set_title('Percentage of leak - 1-minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black', )
ax1.set_ylabel('%', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend('Leak');

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_leak_1min_median.leak_pc.plot(ax = ax1, color = 'blue', 
                                   ylim = [0, 50], )
ax1.set_title('Percentage of leak - 1-minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black', )
ax1.set_ylabel('%', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend('Leak');

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_leak_1min_IQR.leak_pc.plot(ax = ax1, color = 'blue', 
                                   ylim = [0, 50], )
ax1.set_title('Percentage of leak - 1-minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black', )
ax1.set_ylabel('%', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend('Leak');

## Flow

#### 1/sec data, not resampled

#### resampled over 1-min time windows 

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_flow_1min_median.FlowDev.plot(ax = ax1, color = 'blue', ylim = [0, (slow_Measurement_flow_1min_median.FlowDev.max() * 1.2)])
ax1.set_title('Ventilator flow - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Flow (L/min)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['Ventilator Flow' ]);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_flow_1min_IQR.FlowDev.plot(ax = ax1, color = 'blue', ylim = [0, 
                                                     (slow_Measurement_flow_1min_IQR.FlowDev.max() * 1.2)])
ax1.set_title('Ventilator flow - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Flow (L/min)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['Ventilator Flow' ]);

## I : E ratios

#### Resampled over 1-min time windows

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_IE_1min_median.IE_mand.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_IE_1min_median.IE_mand.max() * 1.2)])
ax1.set_title('I:E ratio - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['I : E']);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_IE_1min_IQR.IE_mand.plot(ax = ax1, color = 'red', ylim = [0, (slow_Measurement_IE_1min_IQR.IE_mand.max() * 1.2)])
ax1.set_title('I:E ratio - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['I : E']);

## Compliance, resistance and time constant

#### 1 / sec data, no resampling

#### Resampled over 1-minute windows

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_der_data_1min_median.Cdyn.plot(ax = ax1, color = 'red', 
                                                ylim = [0, (slow_Measurement_der_data_1min_median.Cdyn.max() * 1.2 )])
ax1.set_title('Dynamic compliance - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Compliance (L/bar)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['Cdyn']);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_der_data_1min_IQR.Cdyn.plot(ax = ax1, color = 'red', 
                                                ylim = [0, (slow_Measurement_der_data_1min_IQR.Cdyn.max() * 1.2 )])
ax1.set_title('Dynamic compliance - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Compliance (L/bar)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['Cdyn']);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_der_data_1min_median.R.plot(ax = ax1, color = 'blue', 
                                                ylim = [0, (slow_Measurement_der_data_1min_median.R.max() * 1.2 )])
ax1.set_title('Resistance - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Resistance (mbar/L/sec)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['R']);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_der_data_1min_IQR.R.plot(ax = ax1, color = 'blue', 
                                                ylim = [0, (slow_Measurement_der_data_1min_IQR.R.max() * 1.2 )])
ax1.set_title('Resistance - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Resistance (mbar/L/sec)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['R']);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_der_data_1min_median.TC.plot(ax = ax1, color = 'green', 
                                                ylim = [0, (slow_Measurement_der_data_1min_median.TC.max() * 1.2 )])
ax1.set_title('Time constant - 1 minute medians', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Time constant (sec)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['TC']);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0.7)
ax1 = fig.add_subplot(1, 1, 1);
slow_Measurement_der_data_1min_IQR.TC.plot(ax = ax1, color = 'green', 
                                                ylim = [0, (slow_Measurement_der_data_1min_IQR.TC.max() * 1.2 )])
ax1.set_title('Time constant - 1 minute IQRs', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Time constant (sec)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend(['TC']);

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0)
ax0 = fig.add_subplot(2, 1, 1);
slow_Measurement_pressures_1min_median['PEEP'].plot(ax = ax0, color = 'red', 
                ylim = [-10, (slow_Measurement_pressures_1min_median['PEEP'].max() + 10)], fontsize = 12, xticks = []  );
slow_Measurement_pressures_1min_median['Pmin'].plot(ax = ax0, color = 'green', 
                 ylim = [-10, (slow_Measurement_pressures_1min_median['PEEP'].max() + 10)], xticks = [] );
ax0.set_title('Positive End Expiratory Pressure and Minimum Pressure', size = 16, color = 'black')
ax0.set_xlabel('Time (hours)', size = 14, color = 'black')
ax0.set_ylabel('Pressure (mbar)', size = 14, color = 'black')
ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax0.legend(['PEEP', 'Pmin']);


ax1 = fig.add_subplot(2, 1, 2);
slow_Measurement_MV_1min_median.MVemand_kg.plot(ax = ax1, color = 'red', 
                        ylim = [0, (slow_Measurement_MV_1min_median.MV_kg.max() * 1.2)], fontsize = 12) 
slow_Measurement_MV_1min_median.MVespon_kg.plot(ax = ax1, color = 'green', 
                        ylim = [0, (slow_Measurement_MV_1min_median.MV_kg.max() * 1.2)])
ax1.set_title('Mandatory and Spontaneous Expiratory Minute Volume', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Minute Volume (L/min/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend([ 'MVemand', 'MVespon' ]);

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'Figure_1.jpg'), dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='jpg',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=True)

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0)
ax0 = fig.add_subplot(2, 1, 1);
slow_Measurement_pressures_1min_median['Pmin'].plot(ax = ax0, color = 'black', 
                 ylim = [-10, (slow_Measurement_pressures_1min_median['PEEP'].max() + 10)], xticks = [] );
ax0.set_title('Minimum Pressure', size = 16, color = 'black')
ax0.set_xlabel('Time (hours)', size = 14, color = 'black')
ax0.set_ylabel('Pressure (mbar)', size = 14, color = 'black')
ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax0.legend([ 'Pmin']);


ax1 = fig.add_subplot(2, 1, 2);
slow_Measurement_MV_1min_median.MVemand_kg.plot(ax = ax1, color = 'black', 
                        ylim = [0, (slow_Measurement_MV_1min_median.MV_kg.max() * 1.2)], fontsize = 12) 
ax1.set_title('Mandatory  Expiratory Minute Volume', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Minute Volume (L/min/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend([ 'MVemand' ]);

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'Figure_1_bw.jpg'), dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='jpg',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=True)

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0)
ax0 = fig.add_subplot(2, 1, 1);
slow_Measurement_pressures_1min_median['PEEP'].plot(ax = ax0, color = 'red', 
                ylim = [-10, (slow_Measurement_pressures_1min_median['PEEP'].max() + 10)], fontsize = 12, xticks = []  );
slow_Measurement_pressures_1min_median['Pmin'].plot(ax = ax0, color = 'green', 
                 ylim = [-10, (slow_Measurement_pressures_1min_median['PEEP'].max() + 10)], xticks = [] );
ax0.set_title('Positive End Expiratory Pressure and Minimum Pressure', size = 16, color = 'black')
ax0.get_xaxis().set_visible(False)
ax0.set_xlabel('')
ax0.set_ylabel('Pressure (mbar)', size = 14, color = 'black')
ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax0.legend(['PEEP', 'Pmin']);


ax1 = fig.add_subplot(2, 1, 2);
slow_Measurement_MV_1min_median.MVe_kg.plot(ax = ax1, color = 'blue', 
                        ylim = [0, (slow_Measurement_MV_1min_median.MVe_kg.max() * 1.2)], fontsize = 12) 
ax1.set_title('Expiratory Minute Volume', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (L/min/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend([ 'MV' ]);

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'Figure_1_v2.jpg'), dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='jpg',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=True)

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,
wspace=None, hspace=0)
ax0 = fig.add_subplot(2, 1, 1);
slow_Measurement_pressures_1min_median['PEEP'].plot(ax = ax0, color = 'black', linestyle = 'dotted',
                ylim = [-10, (slow_Measurement_pressures_1min_median['PEEP'].max() + 10)], fontsize = 12, xticks = []  );
slow_Measurement_pressures_1min_median['Pmin'].plot(ax = ax0, color = 'black', 
                 ylim = [-10, (slow_Measurement_pressures_1min_median['PEEP'].max() + 10)], xticks = [] );
ax0.set_title('Positive End Expiratory Pressure and Minimum Pressure', size = 16, color = 'black')
ax0.get_xaxis().set_visible(False)
ax0.set_xlabel('')
ax0.set_ylabel('Pressure (mbar)', size = 14, color = 'black')
ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax0.legend(['PEEP', 'Pmin']);


ax1 = fig.add_subplot(2, 1, 2);
slow_Measurement_MV_1min_median.MVe_kg.plot(ax = ax1, color = 'black', 
                        ylim = [0, (slow_Measurement_MV_1min_median.MVe_kg.max() * 1.2)], fontsize = 12) 
ax1.set_title('Expiratory Minute Volume', size = 16, color = 'black')
ax1.set_xlabel('Time (hours)', size = 14, color = 'black')
ax1.set_ylabel('Volume (L/min/kg)', size = 14, color = 'black')
ax1.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax1.legend([ 'MVe' ]);

fig.savefig('%s/%s%s' % (dir_write, fnames2, 'Figure_1_v2_bw.jpg'), dpi=600, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format='jpg',
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=True)

In [ ]:
slow_Measurement_MV_1min_median.MVe_kg.plot?